# Caten: Python Bindings for ISL - Tutorial


Caten provides Python bindings for the Integer Set Library (ISL). It features automatic memory management, comprehensive type hints, and a Pythonic API.


## 1. Setup and Implicit Context


You can start using ISL objects immediately. The library context is managed automatically, so you don't need to explicitly allocate or free it.


In [ ]:
import caten.isl as I

# You can verify the context is working by creating an object
s = I.Set("{ [i] : 0 <= i < 10 }")
print(f"Set created: {s}")


## 2. Sets and Basic Operations


Sets (`isl_set`) are the core data structure in ISL. They represent sets of integer tuples defined by affine constraints.


In [ ]:
# Create sets from string representations
s1 = I.Set("{ [i] : 0 <= i < 10 }")
s2 = I.Set("{ [i] : 5 <= i < 15 }")

print(f"s1: {s1}")
print(f"s2: {s2}")

# Intersection
s_inter = s1.intersect(s2)
print(f"Intersection (s1 ∩ s2): {s_inter}")

# Union
s_union = s1.union(s2)
print(f"Union (s1 ∪ s2): {s_union}")

# Difference
s_diff = s1.subtract(s2)
print(f"Difference (s1 - s2): {s_diff}")

# Check properties
print(f"Is intersection empty? {s_inter.is_empty()}")
print(f"Is s1 a subset of s_union? {s1.is_subset(s_union)}")


## 3. Maps (Relations)


Maps (`isl_map`) represent relations between integer tuples. They map a domain set to a range set.


In [ ]:
# Create a map
m = I.Map("{ [i] -> [i+1] : 0 <= i < 10 }")
print(f"Map: {m}")

# Apply map to a set
# Image of s1 under m: { y | exists x in s1 s.t. (x, y) in m }
s_image = s1.apply(m)
print(f"Image of s1 under m: {s_image}")

# Intersection of domain
m_restricted = m.intersect_domain(I.Set("{ [i] : i < 5 }"))
print(f"Restricted Map: {m_restricted}")


## 4. Spaces and Identifiers


Spaces (`isl_space`) define the dimensionality and structure of sets and maps.


In [ ]:
# Create a set space with 0 parameters and 2 dimensions
space = I.Space.set_alloc(0, 2) 

# Set tuple name
space = space.set_tuple_name(I.ISLDimType.ISL_DIM_SET, "Point")
print(f"Space: {space}")

# Create an Identifier
id_a = I.Id.alloc("A")
print(f"Identifier: {id_a}")


## 5. Affine Expressions


Affine expressions (`isl_aff`, `isl_multi_aff`, `isl_pw_aff`) represent linear functions.


In [ ]:
# Simple affine expression
aff = I.Aff.from_str("{ [i] -> [2*i + 1] }")
print(f"Aff: {aff}")

# Piecewise affine expression (different expressions for different domains)
pw_aff = I.PwAff.from_str("{ [i] -> [i] : i >= 0; [i] -> [-i] : i < 0 }")
print(f"PwAff (abs): {pw_aff}")

# Multi-affine expression (tuple of affine expressions)
ma = I.MultiAff.from_str("{ [i,j] -> [i+j, i-j] }")
print(f"MultiAff: {ma}")


## 6. Union Types


Union sets (`isl_union_set`) and union maps (`isl_union_map`) can contain relations over different spaces.


In [ ]:
us = I.UnionSet("{ A[i] : 0 <= i < 10; B[j] : 0 <= j < 20 }")
print(f"UnionSet: {us}")

# Operations work similarly to basic sets
us2 = I.UnionSet("{ A[i] : i > 5 }")
print(f"Union Intersection: {us.intersect(us2)}")


## 7. Values


ISL handles arbitrary precision integers using `isl_val`.


In [ ]:
ctx = I.Context.alloc() # Or use implicit context
v = I.Val.int_from_si(42)
print(f"Value: {v}")
print(f"Is positive? {v.sgn() > 0}")

# Operations
v2 = v.add(I.Val.int_from_si(10))
print(f"v + 10 = {v2}")


## 8. Mixin Features and Memory Management


Caten objects inherit from `ISLObjectMixin`, which provides standard Python integrations.


- **Memory Management**: Caten uses `ctypes` and `weakref` to automatically manage ISL objects. You do not need to call `free()`. Objects are freed when they are garbage collected.


- **String Representation**: `str(obj)` calls `isl_..._to_str()` automatically.


- **Equality**: `==` operator calls `isl_..._plain_is_equal()` (or similar).


In [ ]:
# Equality check
s_a = I.Set("{ [i] : i = 0 }")
s_b = I.Set("{ [0] }") # Different syntax, same set semantic

# plain_is_equal checks if the internal representation is identical?
# ISL has is_equal (semantic) and plain_is_equal.
# Caten maps __eq__ to semantic equality if available?
# Let's check semantic equality explicitly if needed.
print(f"Semantic equality (is_equal): {s_a.is_equal(s_b)}")

# repr() shows the string representation
print(f"Repr: {repr(s_a)}")
